In [75]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from astropy.cosmology import FlatLambdaCDM
output_notebook()
%matplotlib notebook

Loading BokehJS ...

In [71]:
hubble_data = np.genfromtxt('hw_2_data/Union2.1RedshiftMag.txt', usecols=(0,1,2,3),
                            names='name, redshift, distance_mod, distance_mod_err',
                            dtype=['O', 'f8', 'f8', 'f8'])
hubble_data = pd.DataFrame(hubble_data)
sample_info = np.genfromtxt('hw_2_data/SCPUnion2.1_AllSNe.txt', delimiter=' & ', usecols=(0,7),
                            names='name, sample_num', dtype=['S12','int'])
sample_info = pd.DataFrame(sample_info)
sample_dict = {1: 'Hamuy et al. (1996)',
               2: 'Krisciunases et al. (2005)',
               3: 'Riess et al. (1999)',
               4: 'Jha et al. (2006)',
               5: 'Kowalski et al. (2008)',
               6: 'Hicken et al. (2009)', 
               7: 'Contreras et al. (2010)',
               8: 'Holtzman et al. (2009)',
               9: 'Riess et al. (1998)',
               10: 'Perlmutter et al. (1999)',
               11: 'Barris et al. (2004)',
               12: 'Amanullah et al. (2008)',
               13: 'Knop et al. (2003)',
               14: 'Astier et al. (2006)',
               15: 'Miknaitis et al. (2007)',
               16: 'Tonry et al. (2003)',
               17: 'Riess et al. (2007)',
               18: 'Amanullah et al. (2010)',
               19: 'Suzuki et al. (2012)'}
sample_info['sample'] = pd.Series([sample_dict[i] for i in sample_info['sample_num']])

merged_data = pd.merge(hubble_data, sample_info)

In [72]:
merged_data[:5]

,name,redshift,distance_mod,distance_mod_err,sample_num,sample
0,b'1993ah',0.028488,35.346583,0.223906,1,Hamuy et al. (1996)
1,b'1993ag',0.050043,36.682368,0.166829,1,Hamuy et al. (1996)
2,b'1993o',0.052926,36.817691,0.155756,1,Hamuy et al. (1996)
3,b'1993b',0.070086,37.446737,0.158467,1,Hamuy et al. (1996)
4,b'1992bs',0.062668,37.483409,0.156099,1,Hamuy et al. (1996)


In [73]:
def errorbar(fig, x, y, xerr=None, yerr=None, point_kwargs={}, error_kwargs={}):
    # Bokeh doesn't have error bars built-in!
    # This function comes from a StackOverflow answer by user MaxNoe
    # http://stackoverflow.com/questions/29166353/how-do-you-add-error-bars-to-bokeh-plots-in-python
    fig.circle(x, y, **point_kwargs)

    if xerr is not None:
        x_err_x = []
        x_err_y = []
        for px, py, err in zip(x, y, xerr):
            x_err_x.append((px - err, px + err))
            x_err_y.append((py, py))
        fig.multi_line(x_err_x, x_err_y, **error_kwargs)

    if yerr is not None:
        y_err_x = []
        y_err_y = []
        for px, py, err in zip(x, y, yerr):
            y_err_x.append((px, px))
            y_err_y.append((py - err, py + err))
        fig.multi_line(y_err_x, y_err_y, **error_kwargs)

In [97]:
cosmo = FlatLambdaCDM(Om0=1-0.729, H0=70)
z = np.linspace(min(merged_data['redshift']), max(merged_data['redshift']), 1000)
distmods = cosmo.distmod(z=z).value

In [98]:
TOOLS = 'wheel_zoom,box_zoom,reset'

p = figure(title='Union 2.1 Compilation Hubble Diagram', tools=TOOLS)
errorbar(p, merged_data['redshift'], merged_data['distance_mod'], yerr=merged_data['distance_mod_err'])
p.line(z, distmods, color='black', line_width=2)
show(p)